In [1]:
import pandas as pd
import sklearn
from sklearn.cross_validation import KFold
import math as m
import numpy as np
from sklearn.cross_validation import train_test_split
from scipy.misc import comb
from sklearn.naive_bayes import MultinomialNB


def calculate_parameters(x,c1,c2,x_test,c11,c22):
    alpha1=[]
    alpha2=[]
    p1_tr=[]
    p2_tr=[]
    p1_te=[]
    p2_te=[]
#     a11=[]
#     a12=[]
#     print x
    for i in range(0,len(c1)):
#         val1=float(sum(x[0:57][i]))
        val1=float(sum(x.loc[i]))
        p1_tr.append(val1)
    for i in range(0,len(c11)):
        val11=float(sum(x_test.loc[i]))
        p1_te.append(val11)
    sum_1 = sum(p1_tr)
    for i in range(57):
        alpha= p1_tr[i]/sum_1
        alpha1.append(alpha)
#     j=0
    for i in range(len(c1),len(x)):
        val2=float(sum(x.loc[i]))
        p2_tr.append(val2)
    for i in range(len(c11),len(x_test)):
        val2=float(sum(x_test.loc[i]))
        p2_te.append(val2)
#         j=j+1
    sum_2 = sum(p2_tr)
    for i in range(57):
        alpha = p2_tr[i]/sum_2 
        alpha2.append(alpha)
    spam_prior= float(len(c1))/float(len(x))
    nospam_prior=float(len(c2))/float(len(x))
    return alpha1, alpha2, spam_prior,nospam_prior, p1_te, p2_te

def member(alpha1, alpha2, spam_prior,nospam_prior,x_test,Y_test,p1_te,p2_te):
    g1=[]
    g2=[]
    val1=[]
    val2=[]
    val11=[]
    val22=[]
    d=pd.DataFrame(columns=('d', 'Label'))
    for i in alpha1:
        if i != 0.0 and i != 1.0:
            val1.append(m.log(i))
            val2.append(m.log(1 - i))
        else:
            val1.append(0.0)
            val2.append(0.0)
    for i in alpha2:
        if i != 0.0 and i != 1.0:
            val11.append(m.log(i))
            val22.append(m.log(1 - i))
        else:
            val11.append(0.0)
            val22.append(0.0)
#     print len(p1_te), len(p2_te)
    g1 = [(np.sum((m.log(comb(p1_te[j],x_test[j][i]))+(x_test[j][i]*val1[j])+((p1_te[j]-x_test[j][i])*val2[j])) for j in range(0,57) ) + m.log(spam_prior)) \
          for i in range(len(x_test))]
    g2 = [(np.sum((m.log(comb(p2_te[j],x_test[j][i]))+(x_test[j][i]*val11[j])+((p2_te[j]-x_test[j][i])*val22[j])) for j in range(0,57)) + m.log(nospam_prior)) \
          for i in range(len(x_test))]
#     print len(g1), len(g2), len(Y_test), len(x_test)
    for i in range(len(x_test)):
        d.loc[i]=[g1[i]-g2[i], Y_test[i]]
    return d

def classify(d,Fun):
    c1=pd.DataFrame(columns=('c1','ALabel', 'PLabel','FunLabel'))
    c2=pd.DataFrame(columns=('c2', 'ALabel','PLabel','FunLabel'))
    for i in range(len(d)):
        if(d.d[i]>=0):
            c1.loc[i]=[d.d[i],d.Label[i],1,Fun[i]]
        else:
            c2.loc[i]=[d.d[i],d.Label[i],0,Fun[i]]
#     print c1
#     print c2
    c=pd.concat([c1,c2])
    return c

def confusion_mat(c):
    TP=0
    FP=0
    TN=0
    FN=0
    for i in range(len(c)):
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==1):
            TP+=1
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==0):
            TN+=1
        if(c.ALabel[i]==0 and c.PLabel[i]==1):
            FP+=1
        if(c.ALabel[i]==1 and c.PLabel[i]==0):
            FN+=1
    con_mat=np.matrix([[TP,FP],[FN,TN]])
    p1= round(float(TP)/float((TP+FP)))
    r1=round(float(TP)/float((TP+FN)))
    p2= round(float(TN)/float((TN+FN)))
    r2=round(float(TN)/float(TN+FP))
    Accuracy=float((TP+TN))/float((TP+FP+TN+FN))
    F1=round(2*(float((p1*r1))/float((p1+r1))))
    F2=round(2*(float((p2*r2))/float((p2+r2))))
    precision=[p1,p2]
    recall=[r1,r2]
    print "Confusion matrix:", con_mat
    print "Accuracy: ",Accuracy
    print "Precision: ",precision
    print "Recall: " ,recall
    print "F_measure: ",[F1,F2]
    
def main():
    filename='spambase.txt'
    rows=[]
    y=[]
    c1=[]
    c2=[]
    c11=[]
    c22=[]
    x=pd.DataFrame(columns=range(57))
    lines = [line.rstrip('\n') for line in open(filename)]
    for i in range(len(lines)):
        row=lines[i].split(',')
        rows.append(row)
        y1=int(rows[i][57])
        y.append(y1)
        x.loc[i]=rows[i][0:57]
        for j in range(len(rows[i])-1):
            x[j][i]= float(x[j][i])
            if(x[j][i]>0):
                x[j][i]=1
            else:
                x[j][i]=0    
    X=np.matrix(x)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    for i in range(len(y_train)):
        if(y_train[i]==1):
            c=y_train[i]
            c1.append(c)
        else:
            c=y_train[i]
            c2.append(c)
    for i in range(len(y_test)):
        if(y_test[i]==1):
            c=y_test[i]
            c11.append(c)
        else:
            c=y_test[i]
            c22.append(c)
    alpha1, alpha2, spam_prior,nospam_prior,p1_te,p2_te=calculate_parameters(X_train,c1,c2,X_test,c11,c22)
    d=member(alpha1, alpha2, spam_prior,nospam_prior,X_test,y_test,p1_te,p2_te)
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    Fun1=clf.predict(np.matrix(X_test))
    c=classify(d,Fun1)
    confusion_mat(c)

if __name__=='__main__':
    main()

Accuracy:  0.466328747285
Precision:  [0.0, 1.0]
Recall:  [1.0, 0.0]
F_measure:  [0.0, 0.0]
